In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torch-1.7.0+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torchvision-0.8.1+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torchaudio-0.7.0-cp37-cp37m-linux_x86_64.whl' --no-deps

In [ ]:
!pip install '/kaggle/input/mmdetectionenv/mmdetectionenv/addict-2.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionenv/mmdetectionenv/yapf-0.32.0-py2.py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionenv/mmdetectionenv/terminal-0.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionenv/mmdetectionenv/terminaltables-3.1.10-py2.py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetection-v2140/mmcv_full-1_3_8-cu110-torch1_7_0/mmcv_full-1.3.8-cp37-cp37m-manylinux1_x86_64.whl' --no-deps


In [ ]:
!pip install '/kaggle/input/pycocotools/pycocotools-2.0.4-cp37-cp37m-linux_x86_64.whl' --no-deps

In [ ]:
!rm -rf mmdetection

!cp -r /kaggle/input/mmdetectionoffline /kaggle/working/
!mv /kaggle/working/mmdetectionoffline /kaggle/working/mmdetection
%cd /kaggle/working/mmdetection

In [ ]:
%cd /kaggle/working/mmdetection/mmdetection


In [ ]:
!pip install -e .

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import sklearn
import torchvision
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
# import cupy as cp
import gc
import pandas as pd
import os
import matplotlib.pyplot as plt
import PIL
import json
from PIL import Image, ImageEnhance
import albumentations as A
import mmdet
import mmcv
from albumentations.pytorch import ToTensorV2
import seaborn as sns
import glob
from pathlib import Path
import pycocotools
from pycocotools import mask
import numpy.random
import random
import cv2
import re
import shutil


In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.apis import inference_detector, init_detector, show_result_pyplot, set_random_seed

In [ ]:
%cd ..

In [ ]:
IMG_WIDTH = 1333
IMG_HEIGHT = 800

In [ ]:
import sys
sys.path.append('/kaggle/input/tensorflow-great-barrier-reef')
import greatbarrierreef

sys.path.append('/kaggle/working/mmdetection')
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

In [ ]:
sys.path.append('/kaggle/input/tensorflow-great-barrier-reef')
env = greatbarrierreef.make_env()
iter_test = env.iter_test() 

In [ ]:
!rm submission.csv

In [ ]:
img_dir = '/kaggle/input/tensorflow-great-barrier-reef/test_images'
config = '/kaggle/input/faster-rcnn-r50-fpn-1x-reef/faster_rcnn_r50_fpn_1x_reef.py'
checkpoint = '/kaggle/input/latestpth/latest.pth'
model = init_detector(config, checkpoint, device='cuda:0')

In [ ]:
def color_correction_of_image_analysis(img):
    """
    基于图像分析的偏色检测及颜色校正方法
    :param img: cv2.imread读取的图片数据
    :return: 返回的白平衡结果图片数据
    """
    b, g, r = cv2.split(img)
    # print(img.shape)
    m, n = b.shape
    # detection(img)
    I_r_2 = np.zeros(r.shape)
    I_b_2 = np.zeros(b.shape)
    I_r_2 = (r.astype(np.float32) ** 2).astype(np.float32)
    I_b_2 = (b.astype(np.float32) ** 2).astype(np.float32)
    sum_I_r_2 = I_r_2.sum()
    sum_I_b_2 = I_b_2.sum()
    sum_I_g = g.sum()
    sum_I_r = r.sum()
    sum_I_b = b.sum()

    max_I_r = r.max()
    max_I_g = g.max()
    max_I_b = b.max()
    max_I_r_2 = I_r_2.max()
    max_I_b_2 = I_b_2.max()
    [u_b, v_b] = np.matmul(np.linalg.inv([[sum_I_b_2, sum_I_b], [max_I_b_2, max_I_b]]), [sum_I_g, max_I_g])
    [u_r, v_r] = np.matmul(np.linalg.inv([[sum_I_r_2, sum_I_r], [max_I_r_2, max_I_r]]), [sum_I_g, max_I_g])
    b_point = u_b * (b.astype(np.float32) ** 2) + v_b * b.astype(np.float32)
    r_point = u_r * (r.astype(np.float32) ** 2) + v_r * r.astype(np.float32)
    b_point[b_point > 255] = 255
    b_point[b_point < 0] = 0
    b = b_point.astype(np.uint8)
    r_point[r_point > 255] = 255
    r_point[r_point < 0] = 0
    r = r_point.astype(np.uint8)
    return cv2.merge([b, g, r])

In [ ]:
score_thr = 0.25
for idx, (img, sample_prediction_df) in enumerate(iter_test):
    print('idx', idx)
    img = img[:, :, ::-1]
    img = color_correction_of_image_analysis(img)
    print(img.shape)
    bboxes = inference_detector(model, img)
    
    predictions = []
    detects = []
    print(bboxes)
    for bbox in bboxes[0]:
        if bbox[-1] >= score_thr:
            xmin, ymin, xmax, ymax = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
            if xmin < 0 or ymin < 0 or xmax > w or ymax > h:
                continue
            predictions.append('{:.2f} {} {} {} {}'.format(bbox[-1], xmin, ymin, xmax-xmin, ymax-ymin))

    prediction_str = ' '.join(predictions)
    print('1111111')
    print(prediction_str)
    sample_prediction_df['annotations'] = prediction_str
    env.predict(sample_prediction_df)

In [ ]:
import pandas as pd
print('2222222')
sub_df = pd.read_csv('submission.csv')
print('3333333')
sub_df.head()
print('4444444')

In [ ]:
!ls /kaggle/working

In [ ]:
# prediction_str
import shutil
!cp  /kaggle/working/mmdetection/submission.csv /kaggle/working/submission.csv

# shutil.rmtree('/kaggle/working/build')
# shutil.rmtree('/kaggle/working/filterpy')
# shutil.rmtree('/kaggle/working/codefixres')
# shutil.rmtree('/kaggle/working/common')
shutil.rmtree('/kaggle/working/mmdetection')
# shutil.rmtree('/kaggle/working/mmpycocotools')
# shutil.rmtree('/kaggle/working/pycocotools')
# shutil.rmtree('/kaggle/working/pycocotools.egg-info')
!ls /kaggle/working

In [ ]:
print(sub_df)